# Run experiments

In [ ]:
decision_makers = ["dot", "eps", "vae", "ssl-rws-count", "ssl-rnd"] # dropout
decision_makers = ["ssl-rnd", "ssl-group"]

In [ ]:
import pickle

In [ ]:
for dm in decision_makers:
    print(f"\tDECISION MODULE {dm}.")

    name = f"ml-{dm}"

    ! python train_ope_offline.py -d moovielens --use_memory --memory_dim 26 -m {dm} --prefix {name} --n_epoch 3 --bs 200 --topk 10 --rws_weight 0.2 --pretrain_predictor

    print(f"\n\n\nEND PROCESSING {dm}, {pm}, {bm}.")
    with open(f"results/{name}.pkl", "rb") as f:
        x = (pickle.load(f))
    print(f"LAST SCORE: {x['replay_ctrs'][-1]}")

In [ ]:
for dm in decision_makers:
    print(f"\tDECISION MODULE {dm}.")

    name = f"lf-{dm}"

    ! python train_ope_offline.py -d lastfm --use_memory --memory_dim 42 -m {dm} --prefix {name} --n_epoch 3 --bs 200 --topk 10 --rws_weight 0.2 --pretrain_predictor

    print(f"\n\n\nEND PROCESSING {dm}, {pm}, {bm}.")
    with open(f"results/{name}.pkl", "rb") as f:
        x = (pickle.load(f))
    print(f"LAST SCORE: {x['replay_ctrs'][-1]}")

# Build tables

In [ ]:
import pandas as pd

metrics_ml = {}
for pm in predictor_modes:
    name = f"results/ml-{dm}"
    with open(f"{name}.pkl", "rb") as f:
        x = (pickle.load(f))
    metrics_ml[dm] = {
        "hitrate": round(x["replay_ctrs"][-1], 4),
        "hitrate_log": x["replay_ctrs"]
    }

In [ ]:
pd.DataFrame(metrics_ml).T.sort_values("hitrate")

In [ ]:
metrics_lf = {}
for pm in predictor_modes:
    name = f"results/lf-{dm}"
    with open(f"{name}.pkl", "rb") as f:
        x = (pickle.load(f))
    metrics_lf[dm] = {
        "hitrate": round(x["replay_ctrs"][-1], 4),
        "hitrate_log": x["replay_ctrs"]
    }

In [ ]:
pd.DataFrame(metrics_lf).T.sort_values("hitrate")

# Plot graphs

In [ ]:
name_map = {
    'dot': "Dot",
    'eps': r"$\varepsilon$-greedy",
    'vae': "VAE",
    'ssl-rws-count': "SSL PPR",
    'ssl-rnd': "SSL RND",
}

In [ ]:
cap = min([len(metrics_lf[dm]["hitrate_log"]) for dm in decision_makers])

for dm in decision_makers:
    plt.plot(metrics_lf[dm]["hitrate_log"][:cap], label=name_map.get(dm))
plt.legend(fontsize=16)
plt.show()